In [1]:
%cd ~/school/dl-2024-mp/

/home/nanahoshi/school/dl-2024-mp


In [2]:
import numpy as np
import torch
import os

from torchvision.transforms import (
    CenterCrop,
    Compose,
    Lambda,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

from torch.utils.data import DataLoader

models = os.listdir('OUTPUTS')
models

/home/nanahoshi/bin/miniconda3/envs/joamama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['cifnet-18-tiny-lr0.01-bottleneck',
 'cifnet-18-apple--lr0.001--tbtest',
 'cifnet-18-cucumber-nope--lr0.001--d4_256',
 'cifnet-18-tiny-lr0.01-attention',
 'cifnet-18-tiny_attention--lr0.001--prenorm',
 'cifnet-18-banana--lr0.001--sigmoid_4d_128-128-64-64',
 'cifnet-18-cucumber--lr0.001--d4_256',
 'cifnet-18-tiny-lr0.1-baseline']

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
cifar_test_nolabels = unpickle('explore/cifar_test_nolabels.pkl')
cifar_test_nolabels

{b'data': array([[133, 136, 136, ..., 226, 225, 224],
        [160, 177, 176, ...,  89,  89,  88],
        [255, 255, 255, ..., 211, 213, 215],
        ...,
        [ 29,  29,  45, ..., 156, 155, 154],
        [124, 123, 126, ...,  49,  49,  51],
        [255, 255, 255, ..., 250, 251, 255]], dtype=uint8),
 b'ids': array([   0,    1,    2, ..., 9997, 9998, 9999])}

In [17]:
import torch
import torchvision

from transformers import AutoImageProcessor
from datasets import load_dataset

from models.cifnet import (
    CifNetForImageClassification,
    CifNetConfig,
)

def setup_model(model_name_or_path, ):
    model_config = CifNetConfig.from_pretrained(model_name_or_path)
    # model = CifNetForImageClassification(model_config)
    model = CifNetForImageClassification.from_pretrained(model_name_or_path, config=model_config)
    processor = AutoImageProcessor.from_pretrained(model_name_or_path)
    return model, processor

model_path = os.path.join('OUTPUTS', models[6])
print(model_path)
# model, image_processor = setup_model('OUTPUTS/cifnet-18-cucumber--lr0.001--d4_256', )
model, image_processor = setup_model(model_path)

# If using a GPU
model = model.to('cuda')

OUTPUTS/cifnet-18-cucumber--lr0.001--d4_256


In [18]:
import torch
from torch.utils.data import Dataset
import PIL.Image
class EvalDataset(Dataset):
    def __init__(self, samples, transform):
        self.samples = samples
        self.trasnform = transform

    def __getitem__(self, index):
        sample = self.samples[index]
        sample = sample.reshape((3, 32, 32)).transpose(1, 2, 0)
        sample = PIL.Image.fromarray(sample)
        sample.save('test.jpg')
        sample = self.trasnform(sample)
        return{
            "pixel_values": sample,
            "index": torch.Tensor([index]),
        }
    
    def __len__(self):
        return len(self.samples)


In [19]:
# Preprocessing the datasets

# Define torchvision transforms to be applied to each image.
if "shortest_edge" in image_processor.size:
  size = image_processor.size["shortest_edge"]
else:
  size = (image_processor.size["height"], image_processor.size["width"])
normalize = (
  Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
  if hasattr(image_processor, "image_mean") and hasattr(image_processor, "image_std")
  else Lambda(lambda x: x)
)
# train_transforms = Compose(
#   [
#     RandomResizedCrop(size),
#     RandomHorizontalFlip(),
#     ToTensor(),
#     normalize,
#   ]
# )
# val_transforms = Compose(
#   [
#     Resize(size),
#     CenterCrop(size),
#     ToTensor(),
#     normalize,
#   ]
# )
# dataset_name = "explore/cifar_test_nolabels.csv"
# dataset = load_dataset(dataset_name)
# max_train_samples = 1000
seed = 42
val_transforms = Compose(
  [
    Resize(size),
    CenterCrop(size),
    ToTensor(),
    normalize,
  ]
)

# # Set the validation transforms
# test_dataset = dataset
per_device_test_batch_size = 4
num_workers = 4
test_dataset = EvalDataset(cifar_test_nolabels[b'data'], val_transforms)
# DataLoaders creation:
def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  indexs = torch.stack([example["index"] for example in examples])
  # labels = torch.tensor([example[label_column_name] for example in examples])
  # return {"pixel_values": pixel_values, "labels": labels}
  return {
    "pixel_values": pixel_values,
    "indexs": indexs
  }

# test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=per_device_test_batch_size, num_workers=num_workers,)
test_dataloader = DataLoader(test_dataset, batch_size=per_device_test_batch_size, num_workers=num_workers, collate_fn=collate_fn, shuffle=False)

In [20]:
# preprocessed_test_data = preprocessed_test_data.to('cuda')

ID_list = []
label_list = []
from tqdm import tqdm
for batch in tqdm(test_dataloader):
    pixel_values = batch['pixel_values'].to(model.device)
    indexs = batch['indexs'].flatten().int().tolist()
    # print(indexs)
    # print(pixel_values)
    outputs = model(pixel_values)
    logits = outputs.logits
    labels = torch.argmax(logits, dim=-1).flatten().int().tolist()
    # print(labels)
    # print(indexs)
    ID_list.extend(indexs)
    label_list.extend(labels)
    # break

# Make predictions
# with torch.no_grad():
    # predictions = model(preprocessed_test_data)

# Process predictions as needed (e.g., applying softmax to get probabilities)

100%|██████████| 2500/2500 [00:16<00:00, 155.96it/s]


In [21]:
# to csv
import pandas as pd

# ID,Labels

# Create a DataFrame with the predictions
df = pd.DataFrame({
    'ID': ID_list,
    'Labels': label_list
})

df.to_csv('predictions.csv', index=False)


In [22]:
# check the predictions distribution
df.Labels.value_counts()

Labels
3    1124
9    1078
7    1042
8    1024
1    1007
6     987
5     957
4     951
0     925
2     905
Name: count, dtype: int64

In [25]:
# assign correct labels for each 1 thousand samples, the correct labels are the first 1 thousand labels are in the order of 8 2 9 0 4 3 6 1 7 5
correct_labels = [8, 2, 9, 0, 4, 3, 6, 1, 7, 5]

for i in range(10):
    df.loc[df['ID'].between(i*1000, (i+1)*1000), 'C_Labels'] = correct_labels[i]

In [28]:
# check accuracy of the model
(df['Labels'] == df['C_Labels']).sum() / len(df)

0.8149

In [31]:
# confusion matrix
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(df['C_Labels'], df['Labels'])

In [32]:
cm

array([[758,  23,  29,  21,   2,   9,  12,  16,  79,  51],
       [ 25, 778,   5,   7,   0,   3,   4,   4,  15, 159],
       [ 33,   2, 775,  50,  43,  22,  42,  21,   4,   8],
       [ 13,   0,  19, 808,  10,  96,  23,  16,   7,   8],
       [ 15,   2,  17,  38, 827,  22,  27,  40,  10,   2],
       [  4,   4,  16, 113,  16, 774,  22,  32,  12,   7],
       [ 18,   5,  34,  50,  14,  12, 851,   9,   3,   4],
       [  5,   2,   4,  26,  35,  18,   3, 893,   6,   8],
       [ 45,  15,   5,   9,   3,   0,   3,  10, 882,  28],
       [  9, 176,   1,   2,   1,   1,   0,   1,   6, 803]])